In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import tweepy
#from tweepy import OAuthHandler

#from timeit import default_timer as timer
#import twitter_credentials
import json
import re
import requests
%matplotlib inline

# Gather

###  Gathering Get Request

In [2]:
response = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
if response.status_code == 200:
    print('Success!')
elif response.status_code == 404:
    print('Not Found.')

Success!


In [3]:
images = pd.read_csv('image_predictions.tsv', sep = '\t', encoding = 'utf-8')

In [4]:
df_1 = pd.read_csv('twitter_archive_enhanced.csv',sep = ',')

In [5]:
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_secret)

#api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
#count = 0
#fails_dict = {}
#start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
#with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
#    for tweet_id in tweet_ids:
#        count += 1
#        print(str(count) + ": " + str(tweet_id))
#        try:
#            tweet = api.get_status(tweet_id, tweet_mode='extended')
#            print("Success")
#            json.dump(tweet._json, outfile)
#            outfile.write('\n')
#        except tweepy.TweepError as e:
#            print("Fail")
#            fails_dict[tweet_id] = e
            
#            pass
#end = timer()
#print(end - start)
#print(fails_dict)


In [6]:
#df_2 = pd.DataFrame(columns=['tweet_id','retweet count','favorite_count'])

#with open('tweet_json.txt') as i:
#    for line in i:
#        stat = json.loads(line)
#        tweet_id = stat['id_str']
#        retweet_count = stat['retweet_count']
#        favorite_count = stat['favorite_count']
#        df_2 = df_2.append(pd.DataFrame([[tweet_id,retweet_count,
#                                         favorite_count]], 
#                                       columns=['tweet_id',
#                                        'retweet_count', 'favorite_count']))
#df_2.to_csv (r'C:\Users\tssan\Desktop\Udacity Projects\4wrangle\WeRateDogs.csv',
#                          index = None, header=True, sep='\t', encoding='utf-8')

In [7]:
df_2 = pd.read_csv('WeRateDogs.csv', sep='\t',encoding='utf-8')

# Assessment
### Visual Assessment

In [8]:
df_1.loc[:3].style.applymap(lambda x: 'color: red' if pd.isnull(x) else '')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,nan,nan,2017-08-01 16:23:56 +0000,Twitter for iPhone,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,nan,nan,nan,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,nan,nan,2017-08-01 00:17:27 +0000,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",nan,nan,nan,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,nan,nan,2017-07-31 00:18:03 +0000,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,nan,nan,nan,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,nan,nan,2017-07-30 15:58:51 +0000,Twitter for iPhone,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,nan,nan,nan,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None


In [9]:
images.loc[:3].style.applymap(lambda x: 'color: red' if pd.isnull(x) else '')

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.0614285,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.0741917,True,Rhodesian_ridgeback,0.07201,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True


In [10]:
df_2.loc[:3].style.applymap(lambda x: 'color: red' if pd.isnull(x) else '')

,favorite_count,retweet count,retweet_count,tweet_id
0,37135,nan,7937,892420643555336193
1,31939,nan,5900,892177421306343426
2,24054,nan,3899,891815181378084864
3,40456,nan,8095,891689557279858688


### Programatic Assessment

In [11]:
df_1.count()

tweet_id                      2356
in_reply_to_status_id           78
in_reply_to_user_id             78
timestamp                     2356
source                        2356
text                          2356
retweeted_status_id            181
retweeted_status_user_id       181
retweeted_status_timestamp     181
expanded_urls                 2297
rating_numerator              2356
rating_denominator            2356
name                          2356
doggo                         2356
floofer                       2356
pupper                        2356
puppo                         2356
dtype: int64

- Five columns in the **df_1** Dataframe appear to contain missing values: 
    - **in_reply_to_status_id**, 
    - **in_reply_to_user_id**, 
    - **retweeed_status_id**, 
    - **retweeted_status_user_id**,  
    - **retweeted_status_timestamp** 


- As observed above:
    - **in_reply_to_status_id** and **in_reply_to_user_id** each have 2247 null values
    - **retweeed_status_id**, **retweeted_status_user_id**, and **retweeted_status_timestamp** each have 2094 null values


- The **text** column contains the body of the tweet, but then ends each tweet with a hyperlink.  The hyperlink is already represented in the **expanded_urls** column, so having it shown twice is redundant information and is not really part of the text of the tweet. 


- The columns **doggo**, **floofer**, **pupper**, and **puppo** have the string "None" in many of there rows. These columns seem to be categorical varibales and would benefit by finding a melt them into a single categorical column.


- The **timestamp** column is formatted down to the millisecond. This is accurate and thourough information, but for future timeseries analysis, I will convert the time in a date format of yyyy/MM/dd HH:mm instead of the yyyy-MM-dd HH:mm:ss +ffff it is in now.



In [12]:
images.count()

tweet_id    2075
jpg_url     2075
img_num     2075
p1          2075
p1_conf     2075
p1_dog      2075
p2          2075
p2_conf     2075
p2_dog      2075
p3          2075
p3_conf     2075
p3_dog      2075
dtype: int64

- The columns **p1**, **p2**, and **p3** are all in snake case format and some of the categorical values are capitalized while others are not.  To clean this up I would like to remove the underscore(s) of each row and put the classications in title format 

In [13]:
df_2.count()

favorite_count    2166
retweet count        0
retweet_count     2166
tweet_id          2166
dtype: int64

**retweet count** is an inconsistent format by not having an underscore to replace the space between words. By grouping column names in snake case the column names of a dataframe are callable. This would likely be a column name that I renamed to much python convention for a column name but it looks like it is a duplicate column to **reteet_count** and has a majority of NaN values. I will drop the column **retweet count** to tidy up the **df_2** DataFrame.

##### Checking for Duplicate Rows

In [14]:
df_1.tweet_id.duplicated().any(), df_2.tweet_id.duplicated().any(), images.tweet_id.duplicated().any()

(False, True, False)

**df_2** contains some duplicated rows that will need to be dropped. 

In [15]:
doggo = df_1.doggo.value_counts()
floofer = df_1.floofer.value_counts()
pupper = df_1.pupper.value_counts()
puppo = df_1.puppo.value_counts()

print(f'{doggo},\n\n{floofer},\n\n{pupper},\n\n{puppo}')

None     2259
doggo      97
Name: doggo, dtype: int64,

None       2346
floofer      10
Name: floofer, dtype: int64,

None      2099
pupper     257
Name: pupper, dtype: int64,

None     2326
puppo      30
Name: puppo, dtype: int64


In [16]:
df_1.rating_numerator.value_counts().sort_index()

0         2
1         9
2         9
3        19
4        17
5        37
6        32
7        55
8       102
9       158
10      461
11      464
12      558
13      351
14       54
15        2
17        1
20        1
24        1
26        1
27        1
44        1
45        1
50        1
60        1
75        2
80        1
84        1
88        1
99        1
121       1
143       1
144       1
165       1
182       1
204       1
420       2
666       1
960       1
1776      1
Name: rating_numerator, dtype: int64

#### Defining a funtion printOut as a visual tool while parsing through the text to get a better look a numerators and denominators

In [17]:
def printOutRating(list1, list2,list3):
    for i in range(len(list1)):
        nex_list1 = list1[i]
        nex_list2 = list2[i]
        nex_list3 = list3[i]
        print(f'Row {nex_list1}\nDenominator:{nex_list2}\n{nex_list3} \n\n')

For most of the cases if a numerator had a value
greater than 14 it was only used once, however 75 occurs twice.
I decided to look into this a little further to figure out why this was.

In [18]:
printout = df_1[df_1.rating_numerator== 75]
        
ind  =  printout.index.tolist()
num  =  printout.rating_numerator.tolist()
text = printout.text.tolist()

printOutRating(ind,num,text)

Row 340
Denominator:75
RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu… 


Row 695
Denominator:75
This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS 




By taking a closer look at these two tweets, I found two problems. The first is that the numerator is not 75, as it is listed in the df_1 dataset, its actually 10. The number 75 was misrepresented as the numerator in error when a rating of 9.75(a sneaky nod to the secret entrance of the Hogwarts Express on platform 9 and 3/4) was given to Logan and second, the indexed tweet #340 happens to be a duplicate of the indexed tweet #695 

In [19]:
df_1.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [20]:
df_1.query('rating_denominator < 10').rating_denominator.count(),df_1.query('rating_denominator > 10').rating_denominator.count()

(3, 20)

2333 of the 2356 observations have a denominator of ten. I decided to look into this further to see if I could find out why this was. 
- 3 tweets have a denominator less than ten.
- 20 tweets have denominators greater than ten. 

In [21]:
printout = df_1[df_1.rating_denominator < 10]
    
ind = printout.index.tolist()
denom = printout.rating_denominator.tolist()
text = printout.text.tolist()

printOutRating(ind,denom,text)

Row 313
Denominator:0
@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho 


Row 516
Denominator:7
Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. 
Keep Sam smiling by clicking and sharing this link:
https://t.co/98tB8y7y7t https://t.co/LouL5vdvxx 


Row 2335
Denominator:2
This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv 




For the the denominators that had values less then ten
- From the the text printout we can see that Row 313 started with 960/00 but was then given the valid rating of 13/10 
- Row 516 was one of the few times where WeRateDogs did not give a rating.
- The tweet in row 2335 misrepresented the rating in error when a fraction of 1/2 was included in the text.

In [22]:
printout = df_1.loc[df_1.rating_denominator > 10]

ind = printout['rating_denominator'].index.tolist()
denom = printout['rating_denominator'].tolist()
text = printout['text'].tolist()

printOutRating(ind,denom,text)

Row 342
Denominator:15
@docmisterio account started on 11/15/15 


Row 433
Denominator:70
The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd 


Row 784
Denominator:11
RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/… 


Row 902
Denominator:150
Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE 


Row 1068
Denominator:11
After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ 


Row 1120
Denominator:170
Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv 


Row 1165
Denominator:20
Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a 


Row 1202
Denominator:50
This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https:

### Summary of Assessment

#### Quality issues
**df_1** dataset
- The columns **in_reply_to_status_id**,**in_reply_to_user_id**,**retweeted_status_id**, **retweeted_status_user_id**,**retweeted_status_timestamp** have almost all NaN values
- The columns Doggo, Pupper, Floofer, and Puppo need to be melted into one column
- Row 313 started with 960/00 but was then given the valid rating of 13/10. These ratings need to be swapped in **df_1** 
- The tweet in row 342 was not a rating, but a reply to question asking for the date that the twitter account began.
- The tweet in row 516 was not given a rating
- Duplicate values need to be dropped 
- Some tweets contain fractions in the body of the text that have been misrepresent as the rating
- Tweets with decimal points show incorrect ratings than what they were actually given
- 

**df_2** dataset
- Duplicate values need to be dropped
- stuff
- stuff

**images** dataset
- No quality errors were found in this dataset

#### Tidiness Issues
**df_1** dataset 
- The dog ratings need a standardized scale in order to facilitate easier comparison. 
- Timestamp would be more interpretable if it were in a format of yyyy/MM/dd HH:mm instead of the yyyy-MM-dd HH:mm:ss +ffff it has in the dataset
- stuff

**df_2** dataset
- stuff
- stuff
- stuff

**images** dataset
- stuff
- stuff


# Clean

In [23]:
df_1_cln = df_1.copy()
df_2_cln = df_2.copy()
images_cln = images.copy()

## Fixing Quality Issue

##### Define
- 1. Drop the columns **in_reply_to_status_id**,**in_reply_to_user_id**,**retweeted_status_id**, **retweeted_status_user_id**,**retweeted_status_timestamp** with NaN values from the **df_1** dataset

##### Code

In [24]:
df_1_cln.drop(columns=['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id',
                   'retweeted_status_user_id','retweeted_status_timestamp'
                   ], inplace=True)

##### Test the results

In [25]:
df_1_cln.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer',
       'pupper', 'puppo'],
      dtype='object')

##### Define 
- Column 'retweet count' has no values, and is a duplicate column name of 'retweet_count'. Because of this, the column should be dropped from **df_2**

##### Code

In [26]:
df_2_cln.drop(columns={'retweet count'},inplace=True)

##### Test the results

In [27]:
if 'retweet count' in df_2_cln.columns :
    print("\nThis column exists in Dataframe")
else :
    print("\nThis column does not exists in Dataframe")


This column does not exists in Dataframe


##### Define
- 2. The tweet in row 342 was not a rating, but a reply to question asking for the date that the twitter account began.


In [28]:
df_1_cln.loc[342,:],df_1_cln.loc[342,'text']

(tweet_id                                             832088576586297345
 timestamp                                     2017-02-16 04:45:50 +0000
 source                <a href="http://twitter.com/download/iphone" r...
 text                           @docmisterio account started on 11/15/15
 expanded_urls                                                       NaN
 rating_numerator                                                     11
 rating_denominator                                                   15
 name                                                               None
 doggo                                                              None
 floofer                                                            None
 pupper                                                             None
 puppo                                                              None
 Name: 342, dtype: object, '@docmisterio account started on 11/15/15')

#####  Code

In [29]:
df_1_cln.drop([342],inplace=True)

##### Test the results

In [30]:
if 342 in df_1_cln.index :
    print("\nThis value exists in Dataframe")
else :
    print("\nThis value does not exists in Dataframe")


This value does not exists in Dataframe


##### Define
- 3.  Fixing row #516 (No rating given) in the **df_1** dataset

In [31]:
df_1.loc[516,:],df_1.loc[516,'text']

(tweet_id                                                     810984652412424192
 in_reply_to_status_id                                                       NaN
 in_reply_to_user_id                                                         NaN
 timestamp                                             2016-12-19 23:06:23 +0000
 source                        <a href="http://twitter.com/download/iphone" r...
 text                          Meet Sam. She smiles 24/7 &amp; secretly aspir...
 retweeted_status_id                                                         NaN
 retweeted_status_user_id                                                    NaN
 retweeted_status_timestamp                                                  NaN
 expanded_urls                 https://www.gofundme.com/sams-smile,https://tw...
 rating_numerator                                                             24
 rating_denominator                                                            7
 name                       

#####  Code

In [32]:
df_1_cln.loc[516, 'rating_numerator']= np.median(df_1_cln.rating_numerator)
df_1_cln.loc[516, 'rating_denominator']= np.median(df_1_cln.rating_denominator)

In [33]:
print(f"Since Sam was never given a rating, I gave him the median rating which is {df_1_cln.loc[516,'rating_numerator']}/{df_1_cln.loc[516,'rating_denominator']}")

Since Sam was never given a rating, I gave him the median rating which is 11.0/10.0


##### Define
- 4.Dropping the duplicate in the **df_2** dataset

In [34]:
df_2_cln = df_2_cln.drop_duplicates(subset=['tweet_id'],keep='first').reset_index()


##### Test the results

In [35]:
df_2_cln.tweet_id.duplicated().any()

False

##### Define
- 5. Replacing all mislabeled ratings with the correct rating

In [36]:
indices = []
for i, row in df_1_cln.iterrows():
    if df_1_cln.rating_denominator[i] != 10:
        in_str= row['text']
        try:
            test = re.search(r'\d+(?:\.\d+)?/10', in_str).group()
            den = re.findall(r'\d+', test)[1]
            test = re.findall(r'\d+', test)[0]
            df_1_cln.at[i,'rating_numerator'] = test
            df_1_cln.at[i,'rating_denominator'] = den
            indices.append(i)
        except:
            test = 'Unchanged'
        if test == 'Unchanged':
            denominator = 'Unchanged'
        else:
            denominator = den
                
        print(f'Row {i}\nNumerator {test}\nDenominator: {denominator}\n{in_str}\n')
        
    else:
        pass

Row 313
Numerator 13
Denominator: 10
@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho

Row 433
Numerator Unchanged
Denominator: Unchanged
The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd

Row 784
Numerator 14
Denominator: 10
RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…

Row 902
Numerator Unchanged
Denominator: Unchanged
Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE

Row 1068
Numerator 14
Denominator: 10
After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ

Row 1120
Numerator Unchanged
Denominator: Unchanged
Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv

Row 1165
Numerator 13
Denominator: 10
Happy 4/

##### Test the results

In [37]:
for i in range(len(indices)):
    ind = indices[i]
    num = df_1_cln.rating_numerator[indices[i]]
    den = df_1_cln.rating_denominator[indices[i]]
    print(f'Row {ind} has a rating of {num}/{den}')

Row 313 has a rating of 13.0/10.0
Row 784 has a rating of 14.0/10.0
Row 1068 has a rating of 14.0/10.0
Row 1165 has a rating of 13.0/10.0
Row 1202 has a rating of 11.0/10.0
Row 1662 has a rating of 10.0/10.0
Row 2335 has a rating of 9.0/10.0


##### Define
- 6. Normalizing Denominators to a Standard of 10 structuring data to facilitate analysis **df_1**

#####  Code

In [38]:
norm_numerator= []
norm_denominator= []
for i, row in df_1_cln.iterrows():
    norm = 10 / row['rating_denominator']
    num = row['rating_numerator'] = row['rating_numerator']* norm
    norm_numerator.append(num)
    den = row['rating_denominator'] = row['rating_denominator']* norm
    norm_denominator.append(den)

In [39]:
df_1_cln['rating_numerator'] = norm_numerator
df_1_cln['rating_denominator'] = norm_denominator

##### Test the results

In [40]:
df_1_cln.query('rating_denominator !=10')['rating_denominator'].any()

False

In [41]:
df_1_cln.rating_denominator.value_counts()

10.0    2355
Name: rating_denominator, dtype: int64

In [42]:
df_1_cln.rating_numerator.value_counts()

12.0      562
11.0      471
10.0      463
13.0      353
9.0       158
8.0       102
14.0       56
7.0        54
5.0        37
6.0        32
3.0        19
4.0        15
2.0        10
1.0         8
0.0         2
75.0        2
15.0        2
420.0       2
17.0        1
666.0       1
12.5        1
1776.0      1
26.0        1
27.0        1
182.0       1
Name: rating_numerator, dtype: int64

##### Define
- 7.  Correcting Ratings Containing Decimal Places, Using Regular Expression to loop through each text and pull out any digits with decimal places in the **df_1** dataset

#####  Code

In [43]:
for i, row in df_1_cln.iterrows():
    in_str= row['text']
    split = re.split(r'/' , in_str)[0]
    if bool(re.search(r'\d+\.\d+$', split)) == True:
        after = re.findall(r'\d+\.\d+', split)[0]
        before = df_1_cln.loc[i, 'rating_numerator']
        df_1_cln.loc[i, 'rating_numerator'] = after
        print(f"The rating in row {i} used to be {before} and now is {after}.")
    else:
        pass

The rating in row 45 used to be 5.0 and now is 13.5.
The rating in row 340 used to be 75.0 and now is 9.75.
The rating in row 695 used to be 75.0 and now is 9.75.
The rating in row 763 used to be 27.0 and now is 11.27.
The rating in row 1689 used to be 5.0 and now is 9.5.
The rating in row 1712 used to be 26.0 and now is 11.26.


In [44]:
df_1_cln.rating_numerator.dtype

dtype('O')

In [45]:
df_1_cln['rating_numerator'] = df_1_cln['rating_numerator'].astype('float64')

##### Test the results

In [46]:
check = df_1_cln.rating_numerator
text = df_1_cln.text
divide = "---" * 38

for i in range(len(check)):
    try:
        chk_dec = check[i]
        txt = text[i] 
        if bool(chk_dec.is_integer()) == False:
            print(f'Now row {i} has the correct rating of {chk_dec}.\n\n{txt}\n{divide}\n')
        else:
            pass
    except:
        pass

Now row 45 has the correct rating of 13.5.

This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948
------------------------------------------------------------------------------------------------------------------

Now row 340 has the correct rating of 9.75.

RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…
------------------------------------------------------------------------------------------------------------------

Now row 695 has the correct rating of 9.75.

This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS
------------------------------------------------------------------------------------------------------------------

Now row 763 has the correct rating of 11.27.

This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. 

##### Define
- 8. 

#####  Code

In [47]:
# Rounding Ratings to get a discrete scale
for i, row in df_1_cln.iterrows():
    before = df_1_cln.loc[i, 'rating_numerator']
    after = round(df_1_cln.rating_numerator[i], 0)
    df_1_cln.loc[i, 'rating_numerator'] = after
    if bool(before == after) == False :
        print(f"The rating in row {i} used to be {before} and now it is {after}.")
    else:
        pass

The rating in row 45 used to be 13.5 and now it is 14.0.
The rating in row 340 used to be 9.75 and now it is 10.0.
The rating in row 695 used to be 9.75 and now it is 10.0.
The rating in row 763 used to be 11.27 and now it is 11.0.
The rating in row 1663 used to be 12.5 and now it is 12.0.
The rating in row 1689 used to be 9.5 and now it is 10.0.
The rating in row 1712 used to be 11.26 and now it is 11.0.


##### Test the results

In [48]:
check = df_1_cln.rating_numerator
tally = []
for i in range(len(check)):
    try:
        chk_dec = check[i]
        chk = chk_dec.is_integer()
        tally.append(not chk)
    except:
        pass
if bool(any(tally)) == True:
    n = len(tally)
    print(f'Some of the {n} tweets failed the test')
else:
    n = len(tally)
    print(f'All {n} tweets passed the test successfully!')

All 2354 tweets passed the test successfully!


##### Define
- 9. 

In [49]:
df_1_cln.rating_numerator.dtype

dtype('float64')

#####  Code

In [50]:
df_1_cln.rating_numerator = df_1_cln.rating_numerator.astype(int) 

##### Test the results

In [51]:
df_1_cln.rating_numerator.dtype

dtype('int32')

##### Define
- 13. The columns **p1**, **p2**, and **p3** are all in snake case format and some of the categorical values are capitalized while others are not.  To clean this up I would like to remove the underscore(s) of each row and put the classications in title format 

##### Code

In [52]:
pvals = ['p1', 'p2', 'p3']

def removeSnake(word):
        return ''.join(x.lower() or '_' for x in word.replace('_', ' '))
for p in pvals: 
    exec(f"images_cln.{p} = images_cln.{p}.apply(lambda x : removeSnake(x))")

##### Test the results

In [53]:
images_cln[['p1','p2','p3']].sample(10)

,p1,p2,p3
730,rottweiler,miniature pinscher,gordon setter
1372,chow,tibetan mastiff,dhole
991,malinois,chesapeake bay retriever,siamese cat
2001,golden retriever,irish terrier,chow
1643,labrador retriever,staffordshire bullterrier,american staffordshire terrier
766,cocker spaniel,golden retriever,airedale
1658,golden retriever,tibetan mastiff,labrador retriever
1957,pembroke,cardigan,basenji
65,toy poodle,chihuahua,bluetick
494,standard schnauzer,miniature schnauzer,sealyham terrier


##### Define
- 10. 

#####  Code

In [54]:
import datetime as dt
try:
    time = pd.to_datetime(df_1_cln['timestamp'])
    df_1_cln['timestamp'] = time.dt.strftime('%Y/%m/%d %H:%M')
    print('Success!')
except:
    print('No changes made')

Success!


##### Test the results

In [55]:
df_1_cln.timestamp.sample(5)

1913    2015/12/08 23:36
638     2016/11/01 02:00
362     2017/02/08 22:00
2353    2015/11/15 23:21
1363    2016/02/25 19:04
Name: timestamp, dtype: object

##### Define
- 'rating_numerator' in **df_1** and 'retweet_count' in **df_2** should both be the data type int32 instead of float64. 

#####  Code

In [56]:
df_1_cln.rating_numerator = df_1_cln.rating_numerator.astype(int)
df_2_cln.retweet_count = df_2_cln.retweet_count.astype(np.int64)

##### Test the results

In [57]:
df_1_cln.rating_numerator.dtype, df_2_cln.retweet_count.dtype

(dtype('int32'), dtype('int64'))

## Fixing Tidiness Issues

This bring us to our first data tidy issue we will be fixing.
1. Combine Numerator and Denominator rows to make a single Rating column
    - We no longer need the **rating_denominator** column since they all are the same value of 10.
    - The column **rating_numerator** can be renamed **rating** and need one value to rate our dogs, on a standardized scale of 10.

In [58]:
df_1_cln.drop(columns=['rating_denominator'],inplace = True)
df_1_cln.rename(columns={"rating_numerator": "rating"},inplace=True)

In [59]:

print(f'Value Counts of new column rating: \n\n{df_1_cln.rating.value_counts()}\n\nUpdated column names to:\n\n {df_1_cln.columns.values}')

Value Counts of new column rating: 

12      563
11      473
10      466
13      353
9       158
8       102
14       57
7        54
5        35
6        32
3        19
4        15
2        10
1         8
420       2
15        2
0         2
17        1
182       1
1776      1
666       1
Name: rating, dtype: int64

Updated column names to:

 ['tweet_id' 'timestamp' 'source' 'text' 'expanded_urls' 'rating' 'name'
 'doggo' 'floofer' 'pupper' 'puppo']


##### Define
- 2. Melting Doggo, Pupper, Floofer, and Puppo down to one category variable named Cute_Name in the **df_1** Dataframe  

#####  Code

In [60]:
a = df_1_cln['doggo']
b = df_1_cln['pupper']
c = df_1_cln['floofer']
d = df_1_cln['puppo']


df_1_cln['cute_name'] = a.str.cat(b.replace('None','')).str.cat(c.replace('None','')).str.cat(d.replace('None',''))

In [61]:
dict_dog = {'Nonefloofer':'floofer','Nonepupper':'pupper','Nonepuppo':'puppo','doggopupper':'doggo, pupper',
           'doggofloofer':'doggo, floofer','doggopuppo':'doggo, puppo','None':'Other'}

for key, value in dict_dog.items():
    df_1_cln['cute_name'] = df_1_cln.cute_name.replace(key,value)
    
df_1_cln.drop(columns=['doggo', 'floofer','pupper', 'puppo'],inplace=True)

##### Test the results

In [62]:
print(f'Value Counts of new column cute_name: \n\n{df_1_cln.cute_name.value_counts()}\n\nUpdated column names to:\n\n {df_1_cln.columns.values}')

Value Counts of new column cute_name: 

Other             1975
pupper             245
doggo               83
puppo               29
doggo, pupper       12
floofer              9
doggo, floofer       1
doggo, puppo         1
Name: cute_name, dtype: int64

Updated column names to:

 ['tweet_id' 'timestamp' 'source' 'text' 'expanded_urls' 'rating' 'name'
 'cute_name']


##### Define

#####  Code

In [63]:
df_1_cln['text'] = df_1_cln.text.apply(lambda text: text.split('http')[0])

##### Test the results

In [64]:
line1 = df_1_cln.text[np.random.randint(0,len(df_1_cln))]
line2 = df_1_cln.text[np.random.randint(0,len(df_1_cln))]
line3 = df_1_cln.text[np.random.randint(0,len(df_1_cln))]

print(f'{line1}\n\n{line2} \n\n{line3}')

This is Willem. He's a Penn State pupper. Thinks the hood makes him more intimidating. It doesn't. 12/10 

This is Bailey. She's rather h*ckin hype for Halloween tomorrow. Carved those pupkins herself. 12/10  

This pupper is sprouting a flower out of her head. 12/10 revolutionary af 


##### Define
- 14. Merging **df_1**, **df_2**, and **images** Dataframes into a single table

In [65]:
len(df_1_cln), len(df_2_cln), len(images_cln)

(2355, 2026, 2075)

#####  Code

In [66]:
df_cln = df_1_cln.merge(df_2_cln, on='tweet_id', how='inner')

In [67]:
df_cln = df_cln.merge(images_cln, on='tweet_id', how='inner')
df_cln = df_cln.set_index('index').reset_index(drop=True)

In [68]:
len(df_cln)

1979

##### Test the results

In [69]:
df_cln.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls', 'rating',
       'name', 'cute_name', 'favorite_count', 'retweet_count', 'jpg_url',
       'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3',
       'p3_conf', 'p3_dog'],
      dtype='object')

In [70]:
df_cln.index

RangeIndex(start=0, stop=1979, step=1)

In [71]:
df_cln.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1979 entries, 0 to 1978
Data columns (total 21 columns):
tweet_id          1979 non-null int64
timestamp         1979 non-null object
source            1979 non-null object
text              1979 non-null object
expanded_urls     1979 non-null object
rating            1979 non-null int32
name              1979 non-null object
cute_name         1979 non-null object
favorite_count    1979 non-null int64
retweet_count     1979 non-null int64
jpg_url           1979 non-null object
img_num           1979 non-null int64
p1                1979 non-null object
p1_conf           1979 non-null float64
p1_dog            1979 non-null bool
p2                1979 non-null object
p2_conf           1979 non-null float64
p2_dog            1979 non-null bool
p3                1979 non-null object
p3_conf           1979 non-null float64
p3_dog            1979 non-null bool
dtypes: bool(3), float64(3), int32(1), int64(4), object(10)
memory usage: 276.5+ K

In [72]:
stop

NameError: name 'stop' is not defined

In [ ]:
df_cln.to_csv (r'~/Desktop/Udacity Projects/4wrangle/WeRateDogs_Clean.csv',
           index = False, header=True, sep='\t', encoding='utf-8')